In [ ]:
import pandas as pd
import numpy as np
import random
import os

# =====================================================
# 1. PEMANGGILAN DATA ASLI
# =====================================================
df_real = pd.read_csv("scrapped_data_honkai_star_rail.csv")

print("📄 Nama kolom dataset asli:")
print(df_real.columns.tolist())
print(f"Jumlah total data asli: {len(df_real)}")

# =====================================================
# 2. DETEKSI KOLOM ULASAN UTAMA
# =====================================================
TEXT_COL = None
for candidate in ['clean_final', 'review', 'content', 'comment', 'text', 'body', 'reviewBody']:
    if candidate in df_real.columns:
        TEXT_COL = candidate
        break

if TEXT_COL is None:
    raise ValueError("❌ Kolom teks ulasan tidak ditemukan. Tentukan kolom teks utama secara manual.")

print(f"✅ Kolom ulasan terdeteksi: '{TEXT_COL}'")

# Hapus data kosong atau hanya berisi spasi
df_real = df_real.dropna(subset=[TEXT_COL])
df_real = df_real[df_real[TEXT_COL].astype(str).str.strip() != ""]

# =====================================================
# 3. IDENTIFIKASI KOLOM UNTUK STRATIFIKASI (JIKA ADA)
# =====================================================
possible_strata = [c for c in ['lang', 'sentiment'] if c in df_real.columns]
if len(possible_strata) > 0:
    print(f"🔹 Kolom stratifikasi digunakan: {possible_strata}")
else:
    print("ℹ️ Tidak ada kolom 'lang' atau 'sentiment'. Sampling dilakukan secara acak.")

# =====================================================
# 4. FUNGSI PEMBUAT DATA DUMMY
# =====================================================
def make_dummy(df, pct=0.05, stratify_cols=None, text_col='review', random_state=42):
    """
    Membuat subset data dummy berdasarkan persentase tertentu.
    Jika kolom stratifikasi tersedia, sampling dilakukan secara proporsional.
    Augmentasi ringan ditambahkan untuk menambah variasi teks.
    """
    random.seed(random_state)
    np.random.seed(random_state)

    n_target = max(1, int(len(df) * pct))
    print(f"\n📊 Membuat dummy {pct*100:.0f}% ({n_target} data)...")

    # Sampling proporsional jika kolom stratifikasi ada
    if stratify_cols and all(c in df.columns for c in stratify_cols):
        df_temp = df.copy()
        df_temp['__key__'] = df_temp[stratify_cols].astype(str).agg('_'.join, axis=1)
        df_sample = (
            df_temp.groupby('__key__', group_keys=False)
            .apply(lambda x: x.sample(frac=pct, replace=len(x)*pct > len(x), random_state=random_state))
            .reset_index(drop=True)
        )
        df_sample.drop(columns=['__key__'], inplace=True, errors='ignore')
    else:
        df_sample = df.sample(n=n_target, random_state=random_state)

    # Tambahkan kolom default bila belum ada
    if 'lang' not in df_sample.columns:
        df_sample['lang'] = 'unknown'
    if 'sentiment' not in df_sample.columns:
        df_sample['sentiment'] = 'unknown'

    # Augmentasi ringan teks (menambah kata acak 20% peluang)
    def augment_text(t):
        t = str(t)
        if random.random() < 0.2:
            t += " " + random.choice(["gacha", "pity", "banner", "turn-based", "update", "lag"])
        return t

    df_sample[text_col] = df_sample[text_col].apply(augment_text)

    print(f"✅ Dummy {pct*100:.0f}% selesai dibuat: {len(df_sample)} baris.")
    return df_sample

# =====================================================
# 5. PEMBUATAN DAN PENYIMPANAN DATA DUMMY
# =====================================================
output_dir = "./"
sizes = [0.05, 0.10, 0.25]  # 5%, 10%, dan 25% dari total data
dummies = {}

for pct in sizes:
    dummy_df = make_dummy(
        df_real,
        pct=pct,
        stratify_cols=possible_strata,
        text_col=TEXT_COL
    )
    out_path = os.path.join(output_dir, f"honkai_dummy_{int(pct*100)}pct.csv")
    dummy_df.to_csv(out_path, index=False, encoding='utf-8')
    dummies[pct] = out_path
    print(f"💾 File disimpan ke: {out_path}")

# =====================================================
# 6. RANGKUMAN
# =====================================================
print("\n🎯 Semua file dummy berhasil dibuat dan disimpan:")
for k, v in dummies.items():
    print(f"- {int(k*100)}% → {v}")


📄 Nama kolom dataset asli:
['userName', 'score', 'at', 'content']
Jumlah total data asli: 25413
✅ Kolom ulasan terdeteksi: 'content'
ℹ️ Tidak ada kolom 'lang' atau 'sentiment'. Sampling dilakukan secara acak.

📊 Membuat dummy 5% (1270 data)...
✅ Dummy 5% selesai dibuat: 1270 baris.
💾 File disimpan ke: ./honkai_dummy_5pct.csv

📊 Membuat dummy 10% (2541 data)...
✅ Dummy 10% selesai dibuat: 2541 baris.
💾 File disimpan ke: ./honkai_dummy_10pct.csv

📊 Membuat dummy 25% (6353 data)...
✅ Dummy 25% selesai dibuat: 6353 baris.
💾 File disimpan ke: ./honkai_dummy_25pct.csv

🎯 Semua file dummy berhasil dibuat dan disimpan:
- 5% → ./honkai_dummy_5pct.csv
- 10% → ./honkai_dummy_10pct.csv
- 25% → ./honkai_dummy_25pct.csv


In [1]:
# ==============================================
# CELL 2: FUNGSI PREPROCESS UNTUK 1 FILE
# ==============================================
def preprocess_file(input_csv: str, output_csv: str):
    """
    Menerapkan pipeline preprocessing (case folding, cleaning,
    normalisasi, tokenizing, stopword removal, stemming)
    ke satu file CSV dan menyimpan hasil ke output_csv.
    """
    print(f"\n=== PREPROCESSING: {input_csv} ===")
    if not os.path.exists(input_csv):
        raise FileNotFoundError(f"File tidak ditemukan: {input_csv}")
    
    df = pd.read_csv(input_csv)
    
    # Deteksi kolom teks utama
    possible_text_cols = ['review', 'content', 'comment', 'text', 'body', 'reviewBody']
    TEXT_COL = next((c for c in possible_text_cols if c in df.columns), None)
    if TEXT_COL is None:
        raise ValueError(f"Tidak ditemukan kolom teks utama di {input_csv}. "
                         f"Kolom yang dicek: {possible_text_cols}")
    
    # Ambil hanya 1 kolom teks dan buang NaN
    df = df[[TEXT_COL]].rename(columns={TEXT_COL: 'content'}).dropna().reset_index(drop=True)
    print(f"  Data dimuat: {len(df)} baris dari kolom '{TEXT_COL}' -> 'content'")
    
    # Jalankan pipeline bertahap
    df["case_folding"] = df["content"].apply(case_folding)
    df["cleaned"] = df["case_folding"].apply(clean_text)
    df["normalized"] = df["cleaned"].apply(normalize_text)
    df["token"] = df["normalized"].apply(tokenize_text)
    df["no_stopword"] = df["token"].apply(remove_stopwords)
    df["stemmed"] = df["no_stopword"].apply(stem_tokens)
    df["clean_final"] = df["stemmed"].apply(lambda x: " ".join(x))
    
    # Filter dokumen benar-benar kosong setelah preprocessing
    before = len(df)
    df["clean_final_length"] = df["clean_final"].str.split().apply(lambda x: len(x) if isinstance(x, list) else len(str(x).split()))
    df = df[df["clean_final"].str.strip() != ""].copy()
    after = len(df)
    df = df.drop(columns=["clean_final_length"])
    
    print(f"  Setelah filter kosong: {after} dokumen tersisa (buang {before - after})")
    
    # Simpan hasil (minimal kolom yang nanti dipakai BERTopic: content, clean_final)
    out_cols = ["content", "clean_final"]
    df[out_cols].to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"  Hasil disimpan ke: {output_csv}")
    
    return df


In [ ]:
# ==============================================
# CELL 1: SETUP PREPROCESSING
# ==============================================
import os
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from typing import List

# Download resource NLTK (jalan sekali saja)
nltk.download('punkt')
nltk.download('stopwords')

# ----------------------------------------------
# Normalisasi kata tidak baku (versi final)
# ----------------------------------------------
normalization_dict = {
    "ga": "tidak", "gak": "tidak", "gaada": "tidak ada", "gacha": "gacha", "pity": "pity", 
    "banner": "banner","bgt": "sekali", "aja": "saja", "jd": "jadi", "yg": "yang", "tpi": "tapi", 
    "bg": "bang", "sy": "saya","gk": "tidak", "dpt": "dapat", "bngt": "sekali", "lg": "lagi",
    "lgi":"lagi", "gw": "saya", "gue": "saya","lu": "kamu","udh": "sudah","udah": "sudah","mksih": "terima kasih", 
    "kalo": "kalau", "kyk": "seperti", "jg": "juga", "blm": "belum", "gmn": "bagaimana", "kek": "seperti",
    "knp": "kenapa", "trs": "terus", "sm": "sama", "cmn": "cuma","blum": "belum", "nyah": "nya",
    "lagih": "lagi","dapet": "dapat", "gem": "game", "relic": "relic", "bnyk": "banyak", 
    "bgtt": "sekali", "baiq": "baik","geme": "game","gim": "game", "heba": "hebat", "rame": "ramai",
    "mantep": "mantap", "muas": "puas", "pls": "please", "ny": "nya", "nitip": "menitip", "gakan": "tidak akan",
    "pw": "power", "tlonk": "tolong", "mw": "mau", "smoga": "semoga", "sma": "sama", "kayak": "seperti",
    "dlu": "dulu", "bgtu": "begitu","tros": "terus", "plis": "please", "gamenya": "game nya", 
    "makasih": "terima kasih", "blom": "belum","seru banget": "sangat seru", "seru bgt": "sangat seru",
    "mantap bgt": "sangat mantap","smua": "semua","adain": "adakan", "adainya": "adakan nya", 
    "dgn": "dengan", "dlm": "dalam", "hrus": "harus", "jgn": "jangan","jga": "juga", "jdi": "jadi", 
    "jgnlah": "janganlah", "kpn": "kapan", "krn": "karena", "lbh": "lebih","mksd": "maksud", 
    "msti": "mesti", "nnti": "nanti", "sblm": "sebelum", "sbg": "sebagai", "sblmnya": "sebelum nya",
    "smg": "semoga", "spt": "seperti", "syr": "saya", "syg": "sayang", "tkt": "takut", "trus": "terus", 
    "tlong":"tolong", "u/": "untuk", "utk": "untuk", "gamplay": "game play", "gameplay": "game play", 
    "banget": "sekali","gemnya": "game nya", "nolongin": "menolong", "nolong": "menolong", 
    "tolongin": "menolong", "kg":"tidak", "kgk": "tidak","kga":"tidak","kagak":"tidak","kaga":"tidak",
    "kagk":"tidak","ngak":"tidak","ngga":"tidak","nggak":"tidak","ngk":"tidak","tdk":"tidak",
    "tdak":"tidak", "trimakasih":"terima kasih", "trmksh":"terima kasih", "trmksih":"terima kasih",
    "ngerilis":"merilis","gg":"good game", "geming": "game","gemink":"game","gamink":"game",
    "hongkai":"honkai", "seruh":"seru","maksih":"terima kasih","dari":"dari", "bosen":"bosan",
    "bacot":"bicara","bacotin":"membicarakan","bacotinlah":"membicarakanlah", "ngotak":"berpikir",
    "ngasih":"memberi", "ngambil":"mengambil", "ngeluarin":"mengeluarkan","ngebug":"terjadi bug", 
    "ngelepasin":"melepaskan", "ngecas":"mengisi","ngebantu":"membantu","mntp":"mantap","ngefix":"memperbaiki",
    "ngeban":"memblokir","ngeblok":"memblokir","ngebanned":"diblokir","banned":"diblokir", "nunggu":"menunggu", 
    "ngumpulin":"mengumpulkan","bnget":"sekali", "bgtu":"begitu","snggat":"sangat","sngt": "sangat",
    "sbenernya":"sebenarnya","sebenernya":"sebenarnya","bgd":"banget", "bgs":"bagus", "kurng":"kurang",
    "krng":"kurang","krang":"kurang","slalu":"selalu", "smakin":"semakin", "syaratnya":"syarat nya", 
    "ketentuanya":"ketentuan nya", "app": "aplikasi","apani": "apa ini", "apaan":"apa", "apanya":"apa nya", 
    "ntar":"nanti", "ntr":"nanti","mager":"malas gerak", "nyerang":"menyerang","sya": "saya", 
    "sy":"saya", "kmu":"kamu", "km":"kamu", "klu":"kalau", "kl":"kalau", "klo":"kalau","klau":"kalau",
    "f p":"free to play","p w":"pay to win","lmyan":"lumayan","mayan":"lumayan","w":"saya",
    "mayanlah":"lumayanlah", "trlalu":"terlalu","ak":"aku","apk":"aplikasi", "pliss":"please",
    "plisss":"please","tlnk":"tolong","tolonk":"tolong", "bikin":"buat","emg":"memang", "gweh":"saya",
    "emang":"memang", "manteb":"mantap", "yng":"yang", "kasi":"kasih", "liat":"lihat", "takkan":"tidak akan",
    "pas":"ketika", "kalok": "kalau","knapa":"kenapa","donlot": "unduh", "knpa":"kenapa","dah":"sudah","dh":"sudah",
    "capek":"lelah","g":"tidak", "matiin":"mematikan","thx":"thanks", "banyakin": "perbanyak",
    "gatau":"tidak tahu", "gk tau":"tidak tahu", "gk tahu":"tidak tahu", "ngapain":"mengapa",
    "sangt":"sangat", "gud":"good", "nyari": "mencari", "nyampe": "sampai", "ngelewatin": "melewati",
    "ku kira":"kukira","ampe":"sampai","tambahin":"tambahkan","buatin":"buatkan", "gampang":"mudah",
    "gampangin":"permudah","gampangan":"kemudahan","bosenin":"membuat bosan","bosenan":"kebosanan",
    "ngelakuin":"melakukan","ngelanjutin":"melanjutkan","lanjutin":"melanjutkan","ngatasin":"mengatasi",
    "dapetin":"mendapatkan", "storynya":"cerita nya", "mantab":"mantap", "mulu":"terus",
}

# ----------------------------------------------
# Stopword list gabungan
# ----------------------------------------------
stop_words_id = set(stopwords.words('indonesian'))
stop_words_en = set(stopwords.words('english'))
additional_stopwords = { 
    "game", "nya", "sih", "aja", "yang", "gw", "gua", "gue", "kak", "min", "makasih", 
    "terimakasih", "keren", "ok", "tolong", "mohon", "banget", "si", "bagus", "keren", 
    "mantap", "gamenya", "okelah", "thanks", "good", "nice", "best", "seru", "oke", 
    "terima", "kasih", "semangat", "moga", "thanks", "thank", "you", "please", "main", 
    "baik", "ya", "sehat", "hoyoverse", "coy", "love", "ni", "suck", "like", "make", "play"                  
}
stop_words = stop_words_id.union(stop_words_en).union(additional_stopwords)

# ----------------------------------------------
# Setup stemmer
# ----------------------------------------------
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# ----------------------------------------------
# Fungsi preprocessing per langkah
# ----------------------------------------------
def case_folding(text: str) -> str:
    return str(text).lower()

def clean_text(text: str) -> str:
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def normalize_text(text: str) -> str:
    words = text.split()
    words = [normalization_dict.get(w, w) for w in words]
    return " ".join(words)

def tokenize_text(text: str) -> List[str]:
    return re.findall(r'\b[^\d\W_]+\b', text)

def remove_stopwords(tokens: List[str]) -> List[str]:
    return [t for t in tokens if t not in stop_words]

def stem_tokens(tokens: List[str]) -> List[str]:
    return [stemmer.stem(t) for t in tokens]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# ==============================================
# CELL 2: FUNGSI PREPROCESS UNTUK 1 FILE
# ==============================================
def preprocess_file(input_csv: str, output_csv: str):
    """
    Menerapkan pipeline preprocessing (case folding, cleaning,
    normalisasi, tokenizing, stopword removal, stemming)
    ke satu file CSV dan menyimpan hasil ke output_csv.
    """
    print(f"\n=== PREPROCESSING: {input_csv} ===")
    if not os.path.exists(input_csv):
        raise FileNotFoundError(f"File tidak ditemukan: {input_csv}")
    
    df = pd.read_csv(input_csv)
    
    # Deteksi kolom teks utama
    possible_text_cols = ['review', 'content', 'comment', 'text', 'body', 'reviewBody']
    TEXT_COL = next((c for c in possible_text_cols if c in df.columns), None)
    if TEXT_COL is None:
        raise ValueError(f"Tidak ditemukan kolom teks utama di {input_csv}. "
                         f"Kolom yang dicek: {possible_text_cols}")
    
    # Ambil hanya 1 kolom teks dan buang NaN
    df = df[[TEXT_COL]].rename(columns={TEXT_COL: 'content'}).dropna().reset_index(drop=True)
    print(f"  Data dimuat: {len(df)} baris dari kolom '{TEXT_COL}' -> 'content'")
    
    # Jalankan pipeline bertahap
    df["case_folding"] = df["content"].apply(case_folding)
    df["cleaned"] = df["case_folding"].apply(clean_text)
    df["normalized"] = df["cleaned"].apply(normalize_text)
    df["token"] = df["normalized"].apply(tokenize_text)
    df["no_stopword"] = df["token"].apply(remove_stopwords)
    df["stemmed"] = df["no_stopword"].apply(stem_tokens)
    df["clean_final"] = df["stemmed"].apply(lambda x: " ".join(x))
    
    # Filter dokumen benar-benar kosong setelah preprocessing
    before = len(df)
    df["clean_final_length"] = df["clean_final"].str.split().apply(lambda x: len(x) if isinstance(x, list) else len(str(x).split()))
    df = df[df["clean_final"].str.strip() != ""].copy()
    after = len(df)
    df = df.drop(columns=["clean_final_length"])
    
    print(f"  Setelah filter kosong: {after} dokumen tersisa (buang {before - after})")
    
    # Simpan hasil (minimal kolom yang nanti dipakai BERTopic: content, clean_final)
    out_cols = ["content", "clean_final"]
    df[out_cols].to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"  Hasil disimpan ke: {output_csv}")
    
    return df


In [4]:
# ==============================================
# CELL 3: SETUP BERTOPIC PIPELINE UNTUK BANYAK DATASET
# ==============================================
import time
import numpy as np
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from collections import Counter

EMB_CACHE_DIR = "emb_cache"
os.makedirs(EMB_CACHE_DIR, exist_ok=True)

LOCAL_INDOBERT_PATH = "./model/indobert-base-p1"
BATCH_SIZE = 32

print("Menyiapkan model embedding IndoBERT...")
if os.path.exists(LOCAL_INDOBERT_PATH):
    print(f"  Menggunakan model lokal: {LOCAL_INDOBERT_PATH}")
    embedding_model = SentenceTransformer(LOCAL_INDOBERT_PATH)
else:
    print("  Mengunduh: indobenchmark/indobert-base-p1")
    embedding_model = SentenceTransformer("indobenchmark/indobert-base-p1")

def auto_param_rules(n_docs: int):
    """
    Param otomatis bergantung ukuran dataset.
    Sama seperti pipeline utama: fokus topik cukup, outlier sedang.
    """
    if n_docs < 3000:
        min_df = 2
        max_df = 0.95
        n_neighbors = 15
        n_components = 5
        min_cluster_size = max(5, int(0.01 * n_docs))
        min_samples = max(5, int(min_cluster_size / 3))
    elif n_docs < 7000:
        min_df = max(2, int(0.002 * n_docs))
        max_df = 0.9
        n_neighbors = 20
        n_components = 5
        min_cluster_size = max(10, int(0.006 * n_docs))
        min_samples = max(5, int(min_cluster_size / 3))
    else:
        min_df = 0.0015
        max_df = 0.8
        n_neighbors = 35
        n_components = 5
        min_cluster_size = max(45, int(0.003 * n_docs))
        min_samples = max(10, int(min_cluster_size / 3))
    return {
        "min_df": min_df,
        "max_df": max_df,
        "n_neighbors": n_neighbors,
        "n_components": n_components,
        "min_cluster_size": min_cluster_size,
        "min_samples": min_samples
    }

def run_bertopic_for_preprocessed(label: str, preprocessed_csv: str, output_root: str = "Hasil_loop"):
    """
    Menjalankan pipeline BERTopic untuk 1 dataset yang SUDAH dipreprocessing
    (harus punya kolom 'clean_final').
    """
    print("\n" + "="*70)
    print(f"RUN BERTOPIC DATASET: {label}")
    print("="*70)
    
    if not os.path.exists(preprocessed_csv):
        print(f"[SKIP] File tidak ditemukan: {preprocessed_csv}")
        return None
    
    df = pd.read_csv(preprocessed_csv, dtype=str)
    if "clean_final" not in df.columns:
        raise ValueError(f"Kolom 'clean_final' tidak ditemukan di {preprocessed_csv}")
    
    docs = df["clean_final"].astype(str).tolist()
    N = len(docs)
    print(f"  Jumlah dokumen: {N}")
    
    params = auto_param_rules(N)
    print("  Parameter otomatis:")
    for k, v in params.items():
        print(f"    {k}: {v}")
    
    # Embeddings cache spesifik per (label, N)
    emb_cache_path = os.path.join(EMB_CACHE_DIR, f"embeddings_{N}.npy")
    if os.path.exists(emb_cache_path):
        embeddings = np.load(emb_cache_path)
        print(f"  Embeddings dimuat dari cache: {emb_cache_path}")
    else:
        print("  Menghitung embeddings...")
        embeddings = embedding_model.encode(docs, batch_size=BATCH_SIZE, show_progress_bar=True)
        np.save(emb_cache_path, embeddings)
        print(f"  Embeddings disimpan ke: {emb_cache_path}")
    
    # Vectorizer
    min_df = params["min_df"]
    max_df = params["max_df"]
    if isinstance(min_df, float):
        vectorizer = CountVectorizer(min_df=min_df, max_df=max_df, ngram_range=(1,2))
    else:
        vectorizer = CountVectorizer(min_df=int(min_df), max_df=max_df, ngram_range=(1,2))
    
    vectorizer.fit(docs)
    vocab_size = len(vectorizer.get_feature_names_out())
    print(f"  Vocabulary size: {vocab_size}")
    if vocab_size == 0:
        print("  [STOP] Vocabulary kosong. Sesuaikan min_df/ngram_range.")
        return None
    
    # UMAP & HDBSCAN
    umap_model = UMAP(
        n_neighbors=params["n_neighbors"],
        n_components=params["n_components"],
        min_dist=0.05,
        metric="cosine",
        random_state=42
    )
    
    hdbscan_model = HDBSCAN(
        min_cluster_size=params["min_cluster_size"],
        min_samples=params["min_samples"],
        metric="euclidean",
        prediction_data=True,
        cluster_selection_method="leaf"
    )
    
    # BERTopic
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer,
        language="multilingual",
        calculate_probabilities=True,
        verbose=False
    )
    
    # Fit
    start = time.time()
    topics, probs = topic_model.fit_transform(docs, embeddings=embeddings)
    elapsed = (time.time() - start) / 60.0
    print(f"  Waktu fit: {elapsed:.2f} menit")
    
    # Ringkasan
    topic_info = topic_model.get_topic_info()
    num_topics = len(topic_info[topic_info["Topic"] != -1])
    outlier_ratio = (np.array(topics) == -1).sum() / len(topics)
    print(f"  Topik efektif: {num_topics}")
    print(f"  Outlier ratio: {outlier_ratio:.2%}")
    
    counts = Counter(topics)
    non_out_counts = {t: c for t, c in counts.items() if t != -1}
    if non_out_counts:
        total_non_out = sum(non_out_counts.values())
        top_pairs = sorted(non_out_counts.items(), key=lambda x: x[1], reverse=True)[:5]
        print("  5 topik terbesar (non-outlier):")
        for tid, c in top_pairs:
            print(f"    Topic {tid}: {c} dokumen ({c/total_non_out*100:.2f}%)")
    
    # Coherence
    tokenized_docs = [d.split() for d in docs]
    dictionary = Dictionary(tokenized_docs)
    coh_scores = []
    for tid in topic_info[topic_info["Topic"] != -1]["Topic"]:
        tid = int(tid)
        count = int(topic_info.loc[topic_info["Topic"] == tid, "Count"].values[0])
        if count < max(5, int(0.002 * N)):
            continue
        words = [w for w, _ in topic_model.get_topic(tid)]
        if len(words) < 2:
            continue
        try:
            cm = CoherenceModel(
                topics=[words],
                texts=tokenized_docs,
                dictionary=dictionary,
                coherence="c_v"
            )
            coh_scores.append(cm.get_coherence())
        except:
            pass
    mean_coh = float(np.mean(coh_scores)) if coh_scores else float("nan")
    print(f"  Mean coherence (c_v): {mean_coh:.4f}")
    
    # Simpan output
    out_dir = os.path.join(output_root, label)
    os.makedirs(out_dir, exist_ok=True)
    
    topic_info.to_csv(os.path.join(out_dir, "topic_info.csv"), index=False, encoding="utf-8-sig")
    df_out = df.copy()
    df_out["pred_topic"] = topics
    df_out.to_csv(os.path.join(out_dir, "docs_with_topic.csv"), index=False, encoding="utf-8-sig")
    
    # evaluasi per topik yang sudah punya skor
    eval_rows = []
    for tid, coh in zip(topic_info[topic_info["Topic"] != -1]["Topic"], coh_scores):
        eval_rows.append({"Topic_ID": int(tid), "Coherence_c_v": coh})
    if eval_rows:
        pd.DataFrame(eval_rows).to_csv(os.path.join(out_dir, "evaluasi_per_topik_c_v.csv"),
                                       index=False, encoding="utf-8-sig")
    
    print(f"  Output disimpan di folder: {out_dir}")
    
    return {
        "label": label,
        "N_docs": N,
        "num_topics": num_topics,
        "outlier_ratio": outlier_ratio,
        "mean_coherence_c_v": mean_coh
    }


Menyiapkan model embedding IndoBERT...
  Menggunakan model lokal: ./model/indobert-base-p1


In [5]:
# ==============================================
# CELL 4: JALANKAN PREPROCESSING + BERTOPIC
# ==============================================

# 1) PREPROCESSING (kalau dummy masih raw)
# Kalau dummy kamu sudah dalam bentuk "dummy_preprocessed_5pct.csv" dkk yang SAMA
# formatnya dengan hasil_preprocessing (punya clean_final), kamu bisa SKIP blok ini
# dan langsung ke langkah run_bertopic_for_preprocessed di bawah.

# Contoh (ganti nama file sesuaikan):
# preprocess_file("scrapped_data_honkai_star_rail.csv", "hasil_preprocessing.csv")
preprocess_file("honkai_dummy_5pct.csv", "dummy_preprocessed_5pct.csv")
preprocess_file("honkai_dummy_10pct.csv", "dummy_preprocessed_10pct.csv")
preprocess_file("honkai_dummy_25pct.csv", "dummy_preprocessed_25pct.csv")

# 2) LOOP BERTOPIC UNTUK DATASET FULL + DUMMY
datasets_preprocessed = [
    # {"label": "full",        "path": "hasil_preprocessing.csv"},
    {"label": "dummy_5pct",  "path": "dummy_preprocessed_5pct.csv"},
    {"label": "dummy_10pct", "path": "dummy_preprocessed_10pct.csv"},
    {"label": "dummy_25pct", "path": "dummy_preprocessed_25pct.csv"},
]

all_results = []
for d in datasets_preprocessed:
    res = run_bertopic_for_preprocessed(d["label"], d["path"], output_root="Hasil_loop")
    if res is not None:
        all_results.append(res)

results_df = pd.DataFrame(all_results)
print("\n================= RINGKASAN SEMUA DATASET =================")
print(results_df)



=== PREPROCESSING: honkai_dummy_5pct.csv ===
  Data dimuat: 1270 baris dari kolom 'content' -> 'content'
  Setelah filter kosong: 1122 dokumen tersisa (buang 148)
  Hasil disimpan ke: dummy_preprocessed_5pct.csv

=== PREPROCESSING: honkai_dummy_10pct.csv ===
  Data dimuat: 2541 baris dari kolom 'content' -> 'content'
  Setelah filter kosong: 2225 dokumen tersisa (buang 316)
  Hasil disimpan ke: dummy_preprocessed_10pct.csv

=== PREPROCESSING: honkai_dummy_25pct.csv ===
  Data dimuat: 6353 baris dari kolom 'content' -> 'content'
  Setelah filter kosong: 5560 dokumen tersisa (buang 793)
  Hasil disimpan ke: dummy_preprocessed_25pct.csv

RUN BERTOPIC DATASET: dummy_5pct
  Jumlah dokumen: 1122
  Parameter otomatis:
    min_df: 2
    max_df: 0.95
    n_neighbors: 15
    n_components: 5
    min_cluster_size: 11
    min_samples: 5
  Embeddings dimuat dari cache: emb_cache\embeddings_1122.npy
  Vocabulary size: 1221
  Waktu fit: 0.60 menit
  Topik efektif: 28
  Outlier ratio: 34.14%
  5 topik